# Linear Solver Performance: Constant Coefficient Poisson, Single Core

### Part 2, Evaluation

In [9]:
#r "BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Platform.LinAlg;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.Grid.RefElements;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();

Using gnuplot: C:\Program Files\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
Databases loaded: 
Capacity: 0
Count: 0



Unhandled exception: System.ApplicationException: Already called.
   at BoSSS.Application.BoSSSpad.BoSSSshell.InitTraceFile() in D:\Users\kummer\Documents\BoSSS-NET5\public\src\L4-application\BoSSSpad\BoSSSshell.cs:line 159
   at BoSSS.Application.BoSSSpad.BoSSSshell.Init() in D:\Users\kummer\Documents\BoSSS-NET5\public\src\L4-application\BoSSSpad\BoSSSshell.cs:line 97
   at Submission#11.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

In [10]:
//var TT = typeof(Microsoft.DotNet.Interactive.Formatting.Formatter);
//foreach(var s in TT.GetMethods().Select(mi => mi.Name).ToArray())
//    Console.Write(s + "   ");

In [11]:
wmg.Init("LinslvPerf_ConstPoissonMpi1");
wmg.SetNameBasedSessionJobControlCorrelation();

Project name is set to 'LinslvPerf_ConstPoissonMpi1'.
Opening existing database '\\dc1\userspace\kummer\bosssdbs-minibatch\LinslvPerf_ConstPoissonMpi1'.


In [12]:
wmg.Sessions

In [13]:
var tempDb = OpenOrCreateDatabase(@"\\fdygitrunner\ValidationTests\LinslvPerf_ConstPoissonMpi1");

Opening existing database '\\fdygitrunner\ValidationTests\LinslvPerf_ConstPoissonMpi1'.


In [14]:
databases

#0: { Session Count = 0; Grid Count = 0; Path = \\dc1\userspace\kummer\bosssdbs-minibatch\LinslvPerf_ConstPoissonMpi1 }
#1: { Session Count = 32; Grid Count = 7; Path = \\fdygitrunner\ValidationTests\LinslvPerf_ConstPoissonMpi1 }


In [15]:
wmg.Sessions

#0: LinslvPerf_ConstPoissonMpi1	SIP_J69120_k3_exp_Kcycle_schwarz	11/04/2021 18:06:45	8666816b...
#1: LinslvPerf_ConstPoissonMpi1	SIP_J163840_k2_exp_Kcycle_schwarz	11/04/2021 18:06:01	f68b6d88...
#2: LinslvPerf_ConstPoissonMpi1	SIP_J163840_k2_exp_gmres_levelpmg	11/04/2021 18:08:08	b976d734...
#3: LinslvPerf_ConstPoissonMpi1	SIP_J20480_k5_exp_gmres_levelpmg	11/04/2021 18:09:39	493fc562...
#4: LinslvPerf_ConstPoissonMpi1	SIP_J20480_k3_classic_pardiso	11/04/2021 18:04:55	7963f80d...
#5: LinslvPerf_ConstPoissonMpi1	SIP_J69120_k2_classic_pardiso	11/04/2021 18:04:27	a4a94934...
#6: LinslvPerf_ConstPoissonMpi1	SIP_J69120_k2_exp_Kcycle_schwarz	11/04/2021 18:05:50	e3f9e9a3...
#7: LinslvPerf_ConstPoissonMpi1	SIP_J69120_k3_exp_gmres_levelpmg	11/04/2021 18:08:57	372e3ccb...
#8: LinslvPerf_ConstPoissonMpi1	SIP_J69120_k2_exp_gmres_levelpmg	11/04/2021 18:07:55	36bd8a75...
#9: LinslvPerf_ConstPoissonMpi1	SIP_J20480_k3_exp_Kcycle_schwarz	11/04/2021 18:06:32	69239995...
#10: LinslvPerf_ConstPoissonMpi1	S

In [16]:
using System.IO;
using BoSSS.Application.SipPoisson;

## Create Table for Post-Processing

In [18]:
// evaluators to add additional columns to the session table
static class AddCols {
    static public object XdgMatrixAssembly_time(ISessionInfo SI) {
        var mcr = SI.GetProfiling()[0];
        var ndS = mcr.FindChildren("MatrixAssembly");
        var nd  = ndS.ElementAt(0);
        return nd.TimeSpentInMethod.TotalSeconds  / nd.CallCount;
    }
    static public object Aggregation_basis_init_time(ISessionInfo SI) {
        var mcr = SI.GetProfiling()[0];
        var ndS = mcr.FindChildren("Aggregation_basis_init");
        var nd  = ndS.ElementAt(0);
        return nd.TimeSpentInMethod.TotalSeconds  / nd.CallCount;
    }
    static public object Solver_Init_time(ISessionInfo SI) {
        var mcr = SI.GetProfiling()[0];
        var ndS = mcr.FindChildren("Solver_Init");
        var nd  = ndS.ElementAt(0);
        //Console.WriteLine("Number of nodes: " + ndS.Count() + " cc " + nd.CallCount );
        return nd.TimeSpentInMethod.TotalSeconds / nd.CallCount;
    }
    static public object Solver_Run_time(ISessionInfo SI) {
        var mcr = SI.GetProfiling()[0];
        var ndS = mcr.FindChildren("Solver_Run");
        var nd  = ndS.ElementAt(0);
        return nd.TimeSpentInMethod.TotalSeconds  / nd.CallCount;
    }
    static public object NoOfCores(ISessionInfo SI){
        return SI.GetProfiling().Length;
        }
}

In [19]:
wmg.AdditionalSessionTableColums.Clear();
wmg.AdditionalSessionTableColums.Add("MatrixAssembly", AddCols.XdgMatrixAssembly_time);
wmg.AdditionalSessionTableColums.Add("Aggregation_basis_init_time", AddCols.Aggregation_basis_init_time);
wmg.AdditionalSessionTableColums.Add("Solver_Init_time", AddCols.Solver_Init_time);
wmg.AdditionalSessionTableColums.Add("Solver_Run_time", AddCols.Solver_Run_time);
wmg.AdditionalSessionTableColums.Add("NoOfCores", AddCols.NoOfCores);

In [25]:
var SessTab = wmg.SessionTable;

//// The Session column can't be serialized,
//// we have to remove it
//List<string> AllCols = FullSessTab.GetColumnNames().ToList();
//AllCols.Remove("Session");
//FullSessTab = FullSessTab.ExtractColumns(AllCols.ToArray());

Select those columns which are of interest:

In [21]:
var SubTab = SessTab.ExtractColumns(
    "SessionName","DGdegree:T", "Grid:NoOfCells", "LinearSolver.SolverCode", "DOFs", "MatrixAssembly",
    "Aggregation_basis_init_time", "Solver_Init_time", "Solver_Run_time", "NoIter");

In [22]:
// Filename
var now         = DateTime.Now;
string docName  = wmg.CurrentProject + "_" + now.Year + "-" + now.Month + "-" + now.Day;
SubTab.SaveToFile(docName + ".json");
SubTab.ToCSVFile(docName + ".csv");

In [ ]:
//SessTab.Print();

## Vizualisation of Results

In [26]:
SessTab.GetColumnNames()

index,value
0,ProjectName
1,SessionName
2,DGdegree:T
3,DGdegree:Tex
4,Bndtype:Dirichlet
5,Bndtype:Neumann
6,NoOfMultigridLevels
7,TimesteppingMode
8,GeneratedFromCode
9,ControlFileText_Index


In [30]:
var SubTab_1 = SubTab.ExtractColumns("SessionName","DGdegree:T", "Grid:NoOfCells", "LinearSolver.SolverCode", "DOFs", "minSolRunT", "maxSolRunT", "NoIter", "RegularTerminated", "Conv","MatrixAssembly","Aggregation_basis_init_time","Solver_Init_time","Solver_Run_time","BlockSize");

Unhandled exception: System.ArgumentException: Column "minSolRunT" does not exist.
   at BoSSS.Application.BoSSSpad.TableExtensions.ExtractColumns(DataTable Tab, String[] ColumnNames) in D:\Users\kummer\Documents\BoSSS-NET5\public\src\L4-application\BoSSSpad\TableExtensions.cs:line 674
   at Submission#32.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

In [28]:
var SubTab_2 = SubTab_1.ExtractRows(delegate(int i, IDictionary<string,object> row) {
    return (Convert.ToBoolean(row["RegularTerminated"]) == true)
          && (Convert.ToInt32(row["Conv"]) != 0);
    });

In [ ]:
var SubTab = SubTab_2.ExtractColumns("SessionName","DGdegree:T", "Grid:NoOfCells", "LinearSolver.SolverCode", "DOFs", "minSolRunT", "maxSolRunT", "NoIter");

In [ ]:
SubTab.Print();

In [ ]:
//Extract to excel-File
SubTab_2.Print();
//SubTab_2.ToCSVFile(WFlowName);

In [ ]:
//extract all Degrees and put them into array
int rowcount          = SubTab.Rows.Count;
string selectedcolumn = "DGdegree:T";
int[] tmp= new int[20];

int count = 0;
for(int r=0;r<rowcount;r++){
System.Data.DataRow row = (System.Data.DataRow)SubTab.Rows[r];
int bla                 = Convert.ToInt32(row[selectedcolumn]);
if (tmp[bla]==0){
  tmp[bla]=1;
  count++;
}
}
int[] DGdegrees = new int[count];
int count2 = 0;
for(int i=0;i<tmp.Length;i++){
   if(tmp[i]==1){
      DGdegrees[count2]=i;
      count2++; 
   }
}
DGdegrees;

In [ ]:
//RuntimePlot.PlotNow();

In [ ]:
//var cl = RuntimePlot.ToGnuplot().PlotCairolatex();
//cl.Preview(trimPage:true);

In [ ]:
//cl.WriteMinimalCompileableExample("C:\\tmp\\mini.tex");

In [ ]:
//using SolverCodes = BoSSS.Solution.Control.LinearSolverConfig.Code;

In [ ]:
int rows    = 3;
int columns = 1;
//LinearSolverCode[] ignore_solvers = {LinearSolverCode.classic_pardiso, LinearSolverCode.classic_cg, LinearSolverCode.exp_softpcg_schwarz, LinearSolverConfig.Code.exp_direct_lapack};
LinearSolverCode[] ignore_solvers ={};
Plot2Ddata[,] multiplots = new Plot2Ddata[rows,columns];
int pDegree = 0;
for(int iRow = 0; iRow < rows; iRow++) {
for(int iCol = 0; iCol < columns; iCol++) {
    
    if(pDegree > rows*columns-1)
        continue;
    int tmpDG = -1;
    if(pDegree < DGdegrees.Length)
        tmpDG = DGdegrees[pDegree];
    
    //Create Graphs
    multiplots[iRow,iCol] = SubTab.ToPlot("DOFs", "minSolRunT", // column for x- and y
       delegate (int iTabRow, 
                 IDictionary<string, object> Row, 
                 out string Nmn, 
                 out PlotFormat Fmt) { 
           // - - - - - - - - - - - - - - - - - - - - - - - - 
           // PlotRowSelector:
           // selects, which table row goes to which graph,
           // and the respective color  
           // - - - - - - - - - - - - - - - - - - - - - - - - 
           int k = Convert.ToInt32(Row["DGdegree:T"]);
           if(k != tmpDG) {
                // degree does not match -> not in this plot
                Nmn = null;
                Fmt = null;
                return;
           }
 
           LinearSolverCode solver_name = (LinearSolverCode)Convert.ToInt32(Row["LinearSolver.SolverCode"]);
           //ignore the solvers specified in ingore_solvers
           foreach(LinearSolverCode sc in ignore_solvers){
               if(solver_name==sc){
                   System.Console.WriteLine("skipped");
                   Nmn = null;
                   Fmt = null;
               return;
               }
           }
           
           // ===!!!=== CAUTION ===!!!===
           // Solverframework (SolverEnum numbers) have changed! Please check LinearSolver and NonLinearSolver for details
           
           //process the other solvers
           Fmt = new PlotFormat();
           switch(solver_name) {
           case LinearSolverCode.classic_pardiso: 
               Nmn           = "Pardiso"; 
               Fmt.PointType = PointTypes.OpenCircle;
               Fmt.DashType  = DashTypes.Dotted;
               break;
           case LinearSolverCode.exp_gmres_levelpmg: 
               Nmn           = "GMRES w. pTG"; 
               Fmt.PointType = PointTypes.Box;
               break;
           case LinearSolverCode.exp_Kcycle_schwarz: 
               Nmn           = "Kcycle w. add.-Schwarz"; 
               Fmt.PointType = PointTypes.LowerTriangle;
               break;
           default:
               Console.WriteLine("unknown: " + solver_name); 
               Nmn = "unknown";
               break;
           } 
           Console.WriteLine("name is: " + solver_name); 
           Fmt.PointSize = 0.5;
           Fmt.Style     = Styles.LinesPoints;
           Fmt.LineColor = LineColors.Black;
           Fmt.LineWidth = 3;
       });
    double[] dof = new[]{1e3,1e6};
    double[] linT = dof.Select(x => x*0.001).ToArray();
    var linP = new Plot2Ddata.XYvalues("linear", dof, linT);
    linP.Format.FromString("- black");
    ArrayTools.AddToArray(linP, ref multiplots[iRow,iCol].dataGroups);
 
    //all about axis
    string Title = string.Format("$k = {0}$", tmpDG);
    multiplots[iRow,iCol].Ylabel = Title;
    multiplots[iRow,iCol].LogX = true;
    multiplots[iRow,iCol].LogY = true;
    //specify range of axis
    multiplots[iRow,iCol].YrangeMin = 1e-2;
    multiplots[iRow,iCol].YrangeMax = 1e+4;
    multiplots[iRow,iCol].XrangeMin = 1e1;
    multiplots[iRow,iCol].XrangeMax = 1e7;
    //multiplots[iRow,iCol].Y2rangeMin = 1e-3;
    //multiplots[iRow,iCol].Y2rangeMax = 1e+4;
    //multiplots[iRow,iCol].X2rangeMin = 1e2;
    //multiplots[iRow,iCol].X2rangeMax = 1e7;
    
    //spacing around plots
    multiplots[iRow,iCol].ShowLegend = false;
    multiplots[iRow,iCol].tmargin = 0;
    multiplots[iRow,iCol].bmargin = 1;
    multiplots[iRow,iCol].lmargin = 10;
    multiplots[iRow,iCol].rmargin = 5;
    multiplots[iRow,iCol].ShowXtics=false;

    //I am legend ...
    if(iRow == 0){
    multiplots[iRow,iCol].ShowLegend = true;
    multiplots[iRow,iCol].LegendAlignment=new string[]{"i","t","l"};
    multiplots[iRow,iCol].LegendSwap=true;
    }

    //and i am special ...
    if(iRow == rows-1)
        multiplots[iRow,iCol].ShowXtics=true;
    pDegree++;
}                        
}

//multiplots.AddDummyPlotsForLegend(3,0);
multiplots.PlotNow()

In [ ]:
//var CL = multiplots.ToGnuplot().PlotCairolatex(xSize:14,ySize:12);

In [ ]:
//CL.SaveTo(Path.Combine(CurrentDocDir, "ConstCoeffPoissonScaling.tex"));

In [ ]:
//CL.WriteMinimalCompileableExample(
//   Path.Combine(CurrentDocDir, "plots", "MainConstCoeffPoissonScaling.tex"),
//   "ConstCoeffPoissonScaling.tex");